#Decission Stump
This `DecisionStump` class implements a weak classifier that makes predictions based on a single feature threshold. It uses polarity to determine whether the classification is based on values above or below the threshold.

In [61]:
import numpy as np

class DecisionStump:
    def __init__(self):
        self.polarity = 1
        self.feature_idx = None
        self.threshold = None
        self.alpha = None

    def predict(self, X):
        n_samples = X.shape[0]
        X_column = X[:, self.feature_idx]
        predictions = np.ones(n_samples)
        if self.polarity == 1:
            predictions[X_column < self.threshold] = -1
        else:
            predictions[X_column > self.threshold] = -1

        return predictions

#AdaBoost

In [62]:
class Adaboost:
    def __init__(self, n_clf=5):
        self.n_clf = n_clf
        self.clfs = []

    def fit(self, X, y):
        n_samples, n_features = X.shape

        # Initialize weights to 1/N
        w = np.full(n_samples, (1 / n_samples))

        self.clfs = []

        # Iterate through classifiers
        for _ in range(self.n_clf):
            clf = DecisionStump()
            min_error = float("inf")

            # greedy search to find best threshold and feature
            for feature_i in range(n_features):
                X_column = X[:, feature_i]
                thresholds = np.unique(X_column)

                for threshold in thresholds:
                    # predict with polarity 1
                    p = 1
                    predictions = np.ones(n_samples)
                    predictions[X_column < threshold] = -1

                    # Error = sum of weights of misclassified samples
                    misclassified = w[y != predictions]
                    error = sum(misclassified)

                    if error > 0.5:
                        error = 1 - error
                        p = -1

                    # store the best configuration
                    if error < min_error:
                        clf.polarity = p
                        clf.threshold = threshold
                        clf.feature_idx = feature_i
                        min_error = error

            # calculate alpha
            EPS = 1e-10
            clf.alpha = 0.5 * np.log((1.0 - min_error + EPS) / (min_error + EPS))

            # calculate predictions and update weights
            predictions = clf.predict(X)

            w *= np.exp(-clf.alpha * y * predictions)
            # Normalize to one
            w /= np.sum(w)

            # Save classifier
            self.clfs.append(clf)

    def predict(self, X):
        clf_preds = [clf.alpha * clf.predict(X) for clf in self.clfs]
        y_pred = np.sum(clf_preds, axis=0)
        y_pred = np.sign(y_pred)

        return y_pred

#Testing with sample dataset

In [67]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split


# Generate a sample dataset using make_classification with more features
X, y = make_classification(n_samples=100, n_features=10, n_informative=5, n_redundant=0,
                            n_classes=2, random_state=42)

#Setting y value to -1 or 1

y[y == 0] = -1

# Split the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



# Create and train the AdaBoost classifier
adaboost = Adaboost(n_clf=50)  # You can adjust the number of classifiers (n_clf)
adaboost.fit(X_train, y_train)

# Predict on the test set
y_pred = adaboost.predict(X_test)


def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

# Calculate accuracy
acc = accuracy(y_test, y_pred)

print("Accuracy:", acc)


Accuracy: 0.8


# sklearn prediction

In [66]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier


# Initialize the decision stump as the base estimator
base_estimator = DecisionTreeClassifier(max_depth=1)  # Decision stump

# Create AdaBoostClassifier with the updated parameter 'estimator'
adaboost_sklearn = AdaBoostClassifier(estimator=base_estimator, n_estimators=5)
adaboost_sklearn.fit(X_train, y_train)

# Make predictions on the test set
y_pred_sklearn = adaboost_sklearn.predict(X_test)

# Calculate the accuracy
accuracy_sklearn = accuracy(y_test, y_pred_sklearn)
print(f"Accuracy (sklearn's AdaBoost): {accuracy_sklearn:.2f}")


Accuracy (sklearn's AdaBoost): 0.90
